<a href="https://colab.research.google.com/github/deep-diver/gpt2-ft-pipeline/blob/main/notebooks/tfrecord_to_gcs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/deep-diver/gpt2-ft-pipeline.git

Cloning into 'gpt2-ft-pipeline'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 71 (delta 12), reused 48 (delta 3), pack-reused 0
Unpacking objects: 100% (71/71), 61.13 KiB | 1.91 MiB/s, done.


In [3]:
!cd gpt2-ft-pipeline/alpaca && tfds build --register_checksums

INFO[build.py]: Loading dataset  from path: /content/gpt2-ft-pipeline/alpaca/alpaca_dataset_builder.py
2023-06-19 06:28:39.036691: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-19 06:28:39.873876: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-19 06:28:40.908831: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "NOT_FOUND: Error executing an HTTP request: HTTP response code 404".
INFO[build.py]: download_and_prepare for dataset alpaca/1.0.0...
INFO[dataset_bu

In [4]:
TARGET_ROOT_DIR = "alpaca"

!mkdir -p {TARGET_TRAIN_DIR}
!mkdir -p {TARGET_TEST_DIR}

In [5]:
import tensorflow_datasets as tfds

builder = tfds.builder("alpaca")
builder.download_and_prepare()

In [6]:
!ls {builder.data_dir}

alpaca-train.tfrecord-00000-of-00001  dataset_info.json
alpaca-val.tfrecord-00000-of-00001    features.json


In [9]:
!cp {builder.data_dir}/alpaca-train.tfrecord-00000-of-00001 {TARGET_ROOT_DIR}/alpaca-train.tfrecord
!cp {builder.data_dir}/alpaca-val.tfrecord-00000-of-00001 {TARGET_ROOT_DIR}/alpaca-val.tfrecord

In [10]:
from google.colab import auth
auth.authenticate_user()

In [11]:
#@title GCS
#@markdown You should change these values as per your preferences. The copy operation can take ~5 minutes.
BUCKET_PATH = "gs://alpaca-tfrecords" #@param {type:"string"}
REGION = "us-central1" #@param {type:"string"}

!gsutil mb -l {REGION} {BUCKET_PATH}
!gsutil -m cp -r {TARGET_ROOT_DIR}/*.tfrecord {BUCKET_PATH}

Creating gs://alpaca-tfrecords/...
You are attempting to perform an operation that requires a project id, with none configured. Please re-run gsutil config and make sure to follow the instructions for finding and entering your default project id.
Copying file://alpaca/alpaca-train.tfrecord [Content-Type=application/octet-stream]...
Copying file://alpaca/alpaca-val.tfrecord [Content-Type=application/octet-stream]...
/ [2/2 files][ 21.2 MiB/ 21.2 MiB] 100% Done                                    
Operation completed over 2 objects/21.2 MiB.                                     
